In [53]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

######Load in functions
sys.path.append("../python/")
import get_counts_inline as gc
# from LL import ll

%load_ext Cython

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [2]:
%%cython

###############################################################################
# LL.pyx
###############################################################################
#
# Evaluate the likelihood as defined in 1705.00009, accounting for the minus
# error. For the moment the prior term has been left out
#
# Mask whatever you want
#
# Written: Nick Rodd, MIT, 5 August 2017
# Modified: Ben Safdi,UM, 15 August 2017
#
###############################################################################


# Import basic functions
import numpy as np
cimport numpy as np
cimport cython


import get_counts_inline as gc
cimport get_counts_inline as gc

DTYPE = np.float
ctypedef np.float_t DTYPE_t

# C functions
cdef extern from "math.h":
    double pow(double x, double y) nogil
    double sqrt(double x) nogil
    double cos(double x) nogil
    double tgamma(double x) nogil
    double exp(double x) nogil
    double sin(double x) nogil
    double fabs(double x) nogil
    double tan(double x) nogil

    double log(double x) nogil
    double lgamma(double x) nogil

cdef double pi = np.pi



@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
cpdef double ll(double[:, :, ::1] PSR_data, double[:, :, ::1] omega_ijk, 
                double Nbulge, double Ndisk, double alpha, double n, 
                double sigma, double z0, double beta_disk, double beta_bulge, double rcut = 3.0, double Lmin = 1.0e31, double Lmax_disk = 1.0e36,double Lmax_bulge = 1.0e36,int Ns = 200, int Nang = 1,double smax_disk = 30) nogil:
    """ Calculate the likelihood as a function of the bulge and disk params
    """

    # Setup loop variables
    cdef double omega_val, Nmodel, Nobs, N_bulge, N_disk
    cdef Py_ssize_t i, j, k
    
    # Now calculate the likelihood over all bins
    cdef double ll = 0.
    for i in range(12): # loop over longitude
        for j in range(12): # loop over latitude
            for k in range(8): # loop over flux
                omega_val = omega_ijk[i,j,k] 

                N_disk = gc.Ndisk_full_ang_ijk(i,j,k,Ndisk, omega_val,n,sigma,z0,beta_disk,Lmin,Lmax_disk,Ns,Nang,smax_disk, 0 )
                N_bulge = gc.Nbulge_full_ang_ijk(i,j,k,Nbulge, omega_val,alpha,beta_bulge,rcut, Lmin,Lmax_bulge,Ns,Nang, 0)

                Nmodel = N_disk #+ N_bulge

                Nobs = PSR_data[i,j,k]

                ll += Nobs*log(Nmodel) - Nmodel #- lgamma(Nobs + 1)
                
    return ll

In [25]:
# Fix base parameters
Nb=0.
n=2.35
sigma=1.528
alpha=2.6
z0=0.7
beta=1.2

In [26]:
# Load the binned efficiency and data files
PSR_data = np.load('../data/PSR_data.npy') + np.load('../data/PSR_data_3fgl.npy')
omega_ijk = np.load('../data/omega_ijk.npy')


In [27]:
%%timeit
ll(PSR_data, omega_ijk, 1, 1, alpha, n, sigma, z0, beta, beta)

10 loops, best of 3: 56.1 ms per loop


## Compare with Nick's LL

In [28]:
sys.path.append("/group/hepheno/smsharma/GCE-2FIG-CrossCheck/likelihood")

In [29]:
from PSR_ll import ll as ll_nick

In [30]:
%%timeit
ll_nick(PSR_data, omega_ijk, 1, 1, alpha, n, sigma, z0, beta)

10 loops, best of 3: 58.6 ms per loop


In [22]:
np.argwhere(np.array(['1','2','3']) == '2')[0][0]

1

In [60]:
sys.path.append("../run/")
from run_class import run_scan

In [61]:
fixed_params = ['n','sigma','alpha','z0','beta_bulge','beta_disk']
fixed_param_vals = [2.35,1.528,2.6,0.7,1.2,1.2]

floated_params = ['N_bulge','N_disk']
floated_param_priors = [[0,1e5],[0,1e5]]

In [62]:
rs = run_scan(fixed_params, fixed_param_vals, floated_params, floated_param_priors)

In [ ]:
rs.perform_scan()

11884.7191334 79394.4656849 [0, 1]
[  1.18847191e+04   7.93944657e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
26002.651453 89222.7292061 [0, 1]
[  2.60026515e+04   8.92227292e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
56196.5465546 88183.1645966 [0, 1]
[  5.61965466e+04   8.81831646e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
68458.0683708 58476.5017033 [0, 1]
[  6.84580684e+04   5.84765017e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
84584.0990543 5240.11850357 [0, 1]
[  8.45840991e+04   5.24011850e+03   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
94649.9526501 58812.5407696 [0, 1]
[  9.46499527e+04   5.88125408e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000

43443.530798 53329.0684223 [0, 1]
[  4.34435308e+04   5.33290684e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
79584.8250389 42735.2547646 [0, 1]
[  7.95848250e+04   4.27352548e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
85556.1673641 88209.438324 [0, 1]
[  8.55561674e+04   8.82094383e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
25706.9349289 47306.2992096 [0, 1]
[  2.57069349e+04   4.73062992e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
27793.6339378 82000.0827312 [0, 1]
[  2.77936339e+04   8.20000827e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
59163.7909412 87736.2072468 [0, 1]
[  5.91637909e+04   8.77362072e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.000000

55044.1920757 97061.0737801 [0, 1]
[  5.50441921e+04   9.70610738e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
56317.5976276 40816.7541027 [0, 1]
[  5.63175976e+04   4.08167541e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
21561.0086918 82006.4067841 [0, 1]
[  2.15610087e+04   8.20064068e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
23510.6170177 67946.7499256 [0, 1]
[  2.35106170e+04   6.79467499e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
68132.3170662 32783.9851379 [0, 1]
[  6.81323171e+04   3.27839851e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
98051.4705181 21046.859026 [0, 1]
[  9.80514705e+04   2.10468590e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000

6552.38628387 31775.3016949 [0, 1]
[  6.55238628e+03   3.17753017e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
12050.2769947 42957.6635361 [0, 1]
[  1.20502770e+04   4.29576635e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
5832.52906799 96514.8568153 [0, 1]
[  5.83252907e+03   9.65148568e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
85678.2138348 36064.3446445 [0, 1]
[  8.56782138e+04   3.60643446e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
48910.3376865 42868.077755 [0, 1]
[  4.89103377e+04   4.28680778e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000000e-01   1.20000000e+00   1.20000000e+00]
77731.9073677 99399.6322155 [0, 1]
[  7.77319074e+04   9.93996322e+04   2.60000000e+00   2.35000000e+00
   1.52800000e+00   7.00000

In [42]:
x = [0,0,0,0,0,0,0]
x[[0,2]] = [3,2]

TypeError: list indices must be integers, not list